In [1]:
pip install ultralytics easyocr opencv-python lxml shapely


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 18.9 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [19]:
import os

BASE_DATASET = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset"

IMG_DIR = os.path.join(BASE_DATASET, "all_images")
XML_DIR = os.path.join(BASE_DATASET, "all_xml")

print("Images:", len(os.listdir(IMG_DIR)))
print("XMLs:", len(os.listdir(XML_DIR)))


Images: 1339
XMLs: 1338


In [9]:
import xml.etree.ElementTree as ET

def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = root.find("filename").text

    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    objects = []
    for obj in root.findall("object"):
        plate_text = obj.find("name").text.strip().upper()

        bndbox = obj.find("bndbox")
        xmin = int(float(bndbox.find("xmin").text))
        ymin = int(float(bndbox.find("ymin").text))
        xmax = int(float(bndbox.find("xmax").text))
        ymax = int(float(bndbox.find("ymax").text))

        objects.append({
            "bbox": [xmin, ymin, xmax, ymax],
            "plate_text": plate_text
        })

    return {
        "filename": filename,
        "image_size": (width, height),
        "objects": objects
    }


In [10]:
sample_xml = os.listdir(XML_DIR)[0]
sample_path = os.path.join(XML_DIR, sample_xml)

data = parse_xml(sample_path)
data


{'filename': 'OD26.jpg',
 'image_size': (272, 363),
 'objects': [{'bbox': [106, 190, 199, 215], 'plate_text': 'OD021481'}]}

In [11]:
gt_data = {}

for xml_file in os.listdir(XML_DIR):
    xml_path = os.path.join(XML_DIR, xml_file)
    parsed = parse_xml(xml_path)

    image_name = parsed["filename"]

    # assume 1 plate per image (true for ANPR)
    if len(parsed["objects"]) == 0:
        continue

    obj = parsed["objects"][0]

    gt_data[image_name] = {
        "bbox": obj["bbox"],
        "plate_text": obj["plate_text"],
        "image_size": parsed["image_size"]
    }

print("Ground truth loaded for:", len(gt_data), "images")


Ground truth loaded for: 1338 images


In [12]:
image_files = set(os.listdir(IMG_DIR))
xml_images = set(gt_data.keys())

matched = image_files & xml_images

print("Images:", len(image_files))
print("XML matched:", len(xml_images))
print("Matched:", len(matched))


Images: 1339
XML matched: 1338
Matched: 1337


In [5]:
!pip install ultralytics

import os
import cv2
import numpy as np
from ultralytics import YOLO


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [20]:
# Dataset paths
BASE_DATASET = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset"

IMG_DIR = os.path.join(BASE_DATASET, "all_images")
XML_DIR = os.path.join(BASE_DATASET, "all_xml")

# YOLO model
YOLO_WEIGHTS = "/content/drive/MyDrive/ANPR_Model_Full/weights/best.pt"

model = YOLO(YOLO_WEIGHTS)



In [14]:
def compute_iou(boxA, boxB):
    """
    boxA, boxB: [xmin, ymin, xmax, ymax]
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    inter_w = max(0, xB - xA)
    inter_h = max(0, yB - yA)
    inter_area = inter_w * inter_h

    boxA_area = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxB_area = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    union_area = boxA_area + boxB_area - inter_area

    if union_area == 0:
        return 0.0

    return inter_area / union_area


In [21]:
results_log = []

detected_count = 0
total_images = 0
iou_values = []

for image_name, gt in gt_data.items():
    img_path = os.path.join(IMG_DIR, image_name)
    if not os.path.exists(img_path):
        continue

    image = cv2.imread(img_path)
    total_images += 1

    # YOLO inference
    preds = model(image, conf=0.25, iou=0.5, verbose=False)[0]

    if preds.boxes is None or len(preds.boxes) == 0:
        results_log.append({
            "image": image_name,
            "detected": False,
            "iou": 0.0,
            "error": "DET_MISS"
        })
        continue

    # Take best detection (highest confidence)
    boxes = preds.boxes.xyxy.cpu().numpy()
    scores = preds.boxes.conf.cpu().numpy()
    best_idx = np.argmax(scores)
    yolo_box = boxes[best_idx].astype(int).tolist()

    gt_box = gt["bbox"]
    iou = compute_iou(yolo_box, gt_box)

    iou_values.append(iou)
    detected_count += 1

    error_type = None
    if iou < 0.5:
        error_type = "DET_POOR_IOU"

    results_log.append({
        "image": image_name,
        "detected": True,
        "iou": round(iou, 4),
        "error": error_type
    })


In [25]:
import numpy as np

detected_count = sum(1 for r in results_log if r["detected"])
iou_values = [r["iou"] for r in results_log if r["detected"]]

recall = detected_count / total_images if total_images else 0
mean_iou = np.mean(iou_values) if iou_values else 0

print("📊 YOLO Detection Metrics")
print("-------------------------")
print(f"Total images     : {total_images}")
print(f"Detected plates  : {detected_count}")
print(f"Recall           : {recall:.4f}")
print(f"Mean IoU         : {mean_iou:.4f}")

📊 YOLO Detection Metrics
-------------------------
Total images     : 0
Detected plates  : 0
Recall           : 0.0000
Mean IoU         : 0.0000


In [43]:
from collections import Counter

error_counter = Counter(
    [r["error"] for r in results_log if r["error"] is not None]
)

print("❌ Detection Errors")
for k, v in error_counter.items():
    print(f"{k}: {v}")


❌ Detection Errors


In [ ]:
import json

os.makedirs("/content/drive/MyDrive/ANPR_logs", exist_ok=True)

with open("/content/drive/MyDrive/ANPR_logs/yolo_eval.json", "w") as f:
    json.dump(results_log, f, indent=2)

print("YOLO evaluation log saved")


In [27]:
print("image_names length:", len(image_names))


NameError: name 'image_names' is not defined